In [ ]:
import os
os.getcwd()

## 1.88 wildfire dataset

In [ ]:
# https://www.kaggle.com/datasets/rtatman/188-million-us-wildfires/code?select=FPA_FOD_20170508.sqlite

import pandas
import sqlite3

filepath = 'datasets\wildfire_data\FPA_FOD_20170508.sqlite'

# sqlite random testing
con = sqlite3.connect(filepath)
cur = con.cursor()

# execute var
cursor = cur.execute('SELECT * FROM spatial_ref_sys_aux;')
names = list(map(lambda x: x[0], cursor.description))
print(names)

# The result of a "cursor.execute" can be iterated over by row
i = 0
for row in cursor:
    print(row)
    i += 1
    if i > 100:
        break

In [ ]:
#results: kinda a wash, that dataset is confusing

## USA+California Wildfire Data

In [ ]:
# https://www.kaggle.com/datasets/avkashchauhan/california-wildfire-dataset-from-2000-2021?resource=download

import pandas as pd
import sqlite3


filepath = 'datasets/wildfire_data/ca_wildfires/ca_daily_fire_2000_2021-v2.csv'
data = pd.read_csv(filepath)

data1 = data.copy(deep=True)
# print(data.columns)
# data.drop(columns='Date', inplace=True)

In [ ]:
import os
os.getcwd()

In [ ]:
import pandas as pd
%matplotlib inline

data1 = data.copy(deep=True)
print(data1.dtypes)
# convert date to date types
data1['acq_date'] = pd.to_datetime(data1['acq_date'])
# convert objects (categorical) into ordinal types
data1['satellite'] = data1['satellite'].map({'Terra': 1, 'Aqua': 2})
data1['instrument'] = data1['instrument'].map({'MODIS': 1, 'VIIRS':2})

# drop unnecessary data columns
data1.drop(columns=['year', 'month', 'type'], inplace=True)

print(data1.dtypes)
data1


In [ ]:
# scratch
print(data1['frp'].nlargest(100))
# print(data1['instrument'].value_counts())
# data1.plot.hist(by=['frp'], bins=12)
data1['confidence'].mean()
data1['frp'].mean()
data1['frp'].median()
data1.groupby(data1['acq_date'].dt.year)['bright_t31'].mean()
# data1.groupby['bright_t31']
# data1
print(data1.count())
(data1['confidence'] > 70).sum()
# (data1['confidence'] == 60).sum()

# the mean is higher than the median meaning positively skewed
# meaning that there are some HUGE wildfires!

In [ ]:
# I want to reduce columns so I am going to only keep confidence >75
data1 = data1[data1['confidence'] > 80]
data1.count()

# since the data here is not grouped by dates
# am going to aggregate the mean frp for each day
# and have a count of frp as # of wildfires per day
# and also have mean of the bright_t31 values
# and sum the values of the ordinal data (satellites, instruments) and split these into columns based on encoding

# ORRRR, should i just expand the values of the other dataset to fit here and just duplicate them??
# Procedures:
# 1. reduce rows in this dataset to be smaller, so can link to other by date
# 2. expand rows in other dataset to fit this one, so there are many duplicates of the temperature data

# data1['acq_date'].value_counts()
data1.groupby(data1['acq_date'])['frp'].mean()
data1.groupby(data1['acq_date'])['frp'].count()

In [ ]:
data1

In [ ]:
newdata = data1.nlargest(2000, 'frp')
newdata = newdata.sort_values('acq_date')
extra_df = newdata.groupby('acq_date')['frp'].count()
newdata = newdata.groupby('acq_date').mean().reset_index()
newdata = newdata.merge(extra_df, on='acq_date')
newdata.sort_values('frp_y', ascending = False)


In [ ]:
newdata.groupby(newdata['acq_date'].dt.year)['frp']

In [71]:
# import matplotlib.pyplot as plt
# data1.plot(x="longitude", y="latitude", kind="scatter", c="frp")
newdata

,acq_date,latitude,longitude,satellite,instrument,frp_x,bright_t31,confidence,frp_y
0,2001-01-03,32.802900,-116.689200,1.0,1.0,2284.50,348.1000,100.0,1
1,2001-07-27,40.800400,-120.729850,1.0,1.0,2004.45,318.8000,100.0,2
2,2001-08-11,41.190550,-120.280650,1.0,1.0,2202.00,363.2000,100.0,2
3,2001-08-15,40.580425,-117.668837,1.0,1.0,2708.25,312.1875,100.0,8
4,2001-08-17,40.368800,-117.764800,1.0,1.0,2348.10,305.4000,100.0,1
...,...,...,...,...,...,...,...,...,...
424,2020-10-14,37.480100,-119.168200,2.0,1.0,2133.80,339.7000,87.0,1
425,2020-10-16,37.454700,-119.145400,1.0,1.0,1824.50,311.5000,100.0,1
426,2020-10-19,37.427050,-119.125250,2.0,1.0,1772.65,314.9000,100.0,2
427,2020-10-20,37.426400,-119.109900,2.0,1.0,1874.70,304.6000,100.0,1


In [ ]:
newdata.groupby(newdata['acq_date'].dt.year)['frp_y'].sum()

In [73]:
newdata.to_csv('datasets/outputs/big_wildfires.csv', header=['date', 'latitude', 'longitude', 'satellite', 'instrument', 'avg_frp', 'surface_temp', 'confidence', 'fire_count'])